# FlowForge Chain Design Walkthrough

This notebook shows how to install FlowForge from this repository and design a chain step by step. Run each section in order to mirror the flow you would follow when building your own chain.

## Prerequisites

- Python 3.10+ (written with Python 3.13)
- Run the notebook from the project root so relative imports resolve
- (Optional) activate a virtual environment first

Install FlowForge and optional extras directly from this repo:

```bash
pip install -e .            # core framework
pip install -e .[all]       # include optional summarization/LLM extras
```

If you later consume FlowForge as a package, replace the editable install with `pip install flowforge` once the package is published.

In [ ]:
# Install FlowForge from the repo root (run once per environment).
# Uncomment the extras install if you want LLM/summarization helpers.
!pip install -e .
# !pip install -e .[all]

In [ ]:
import sys
import flowforge

print("Python:", sys.version.split()[0])
print("FlowForge version:", flowforge.__version__)

## 1) Initialize FlowForge

Create an isolated `FlowForge` instance. All steps, chains, and middleware you register here are scoped to this instance.

In [ ]:
from flowforge import ChainContext, FlowForge

forge = FlowForge(name="demo_notebook")

## 2) Define steps

Steps are async functions decorated with `@forge.step`. Use `deps` to declare dependencies and `produces` to document what each step adds to the context.

In [ ]:
from datetime import datetime, timezone


@forge.step(name="collect_context", produces=["context"])
async def collect_context(ctx: ChainContext):
    ctx.set(
        "context",
        {
            "company": "Example Corp",
            "meeting_time": datetime.now(timezone.utc).isoformat(),
            "objective": "Prep for client meeting",
        },
    )
    return {"context": ctx.get("context")}


@forge.step(name="plan_agenda", deps=["collect_context"], produces=["agenda"])
async def plan_agenda(ctx: ChainContext):
    context = ctx.get("context", {})
    agenda = [
        "Review latest company updates",
        "Discuss risk factors and opportunities",
        "Outline next steps with stakeholders",
    ]
    ctx.set("agenda", agenda)
    return {"agenda": agenda, "context_company": context.get("company")}


@forge.step(name="prepare_packet", deps=["plan_agenda"], produces=["prepared_content"])
async def prepare_packet(ctx: ChainContext):
    agenda = ctx.get("agenda", [])
    prepared_content = "\n".join(f"- {item}" for item in agenda)
    ctx.set("prepared_content", prepared_content)
    return {"prepared_content": prepared_content}


## 3) Register the chain

Chains are lightweight classes that list the steps to run. You can provide a custom name and description.

In [ ]:
@forge.chain(name="demo_chain", description="Simple meeting-prep-style chain")
class DemoChain:
    steps = ["collect_context", "plan_agenda", "prepare_packet"]


## 4) Run the chain

Execute the chain and inspect per-step outputs. `forge.run` returns timing, results, and the final context payload.

In [ ]:
import asyncio


async def run_demo():
    result = await forge.run("demo_chain")

    print(f"Success: {result['success']}")
    print(f"Duration (ms): {result['duration_ms']:.2f}")
    for step in result["results"]:
        print(f"{step['step']}: {step['output']}")

    print("\nPrepared content:")
    print(result["context"]["data"]["prepared_content"])


asyncio.run(run_demo())


## 5) Add middleware (optional)

Middleware can add logging, caching, token tracking, or summarization without changing your step code. Attach them to the forge instance, then rerun the chain to see the effect.

In [ ]:
from flowforge.middleware.logger import LoggerMiddleware

# Attach middleware and validate definitions
forge.use_middleware(LoggerMiddleware())
forge.check("demo_chain")

# Rerun to see middleware-driven logging in the notebook output
asyncio.run(run_demo())


## 6) Visualize the DAG

Use the built-in visualizer to render the chain layout in ASCII (works well in notebooks and terminals).

In [ ]:
from flowforge.core.visualize import DAGVisualizer

viz = DAGVisualizer(
    step_registry=forge._step_registry,
    chain_registry=forge._chain_registry,
)
print(viz.to_ascii("demo_chain"))


## 7) Use the prebuilt CMPT chain

The repository ships with a full Client Meeting Prep (CMPT) chain. Provide request details, and FlowForge orchestrates context building, content prioritization, and response generation using the services defined in `flowforge/services`.

Before running it, ensure any required API keys or service configs are set in your environment or config files.

In [ ]:
# Example: run the built-in CMPT chain
from flowforge.chains import CMPTChain

cmpt_chain = CMPTChain()
cmpt_chain.check()

# Replace the sample values with your meeting details and credentials.
# Uncomment asyncio.run(...) when you're ready to execute.
async def run_cmpt():
    result = await cmpt_chain.run(
        corporate_company_name="Apple Inc",
        meeting_datetime="2025-01-15T10:00:00Z",
        corporate_client_names="Finance Team",
        meeting_objective="Quarterly earnings prep",
    )

    print("Prepared content (truncated):")
    print(str(result.response_builder.get("prepared_content"))[:500])


# asyncio.run(run_cmpt())


## 8) Register your own MCP server + chain

Use the MCP connector to wrap your own MCP server as an agent, inject it as a resource, and wire it into a chain. Replace the base URL/tool paths with your server details.


In [ ]:
from flowforge import ChainContext, FlowForge
from flowforge.agents.base import AgentResult
from flowforge.connectors.base import ConnectorConfig
from flowforge.connectors.mcp import MCPAgent

# Configure your MCP endpoint/tool details here
forge_mcp = FlowForge(name="mcp_demo")

@forge_mcp.agent(name="custom_mcp_agent", description="Wraps your MCP server")
class CustomMCPAgent(MCPAgent):
    # Swap base_url/tool endpoints for your MCP server
    connector_config = ConnectorConfig(
        name="my_mcp_server",
        base_url="http://localhost:8000",  # e.g., http://127.0.0.1:8000
        api_key=None,
    )

    async def fetch(self, query: str, **kwargs) -> AgentResult:
        # Example call into MCP; adjust endpoint/payload per your server's tools
        payload = await self.connector.request(
            "/tools/search",
            data={"query": query, **kwargs},
        )

        return AgentResult(
            data=payload,
            source=self.connector_config.name,
            query=query,
        )


async def create_mcp_agent():
    agent = CustomMCPAgent()
    await agent.initialize()
    return agent


async def cleanup_mcp_agent(agent: CustomMCPAgent):
    await agent.cleanup()


# Make the agent available to steps via dependency injection
forge_mcp.register_resource(
    "mcp_agent",
    factory=create_mcp_agent,
    cleanup=cleanup_mcp_agent,
)


## 9) Build a chain that uses your MCP agent

These steps ingest a query, call the MCP agent via the injected resource, and build a response. Replace the logic with your own transforms or merges.


In [ ]:
@forge_mcp.step(name="ingest_request", produces=["mcp_query"])
async def ingest_request(ctx: ChainContext):
    ctx.set("mcp_query", ctx.get("mcp_query", "demo MCP query"))
    return {"mcp_query": ctx.get("mcp_query")}


@forge_mcp.step(
    name="fetch_from_mcp",
    deps=["ingest_request"],
    resources=["mcp_agent"],
    produces=["mcp_raw"],
)
async def fetch_from_mcp(ctx: ChainContext, mcp_agent: CustomMCPAgent):
    result = await mcp_agent.fetch(ctx.get("mcp_query"))
    ctx.set("mcp_raw", result.data)
    return {"mcp_raw": result.data, "mcp_source": result.source}


@forge_mcp.step(
    name="build_custom_response",
    deps=["fetch_from_mcp"],
    produces=["mcp_response"],
)
async def build_custom_response(ctx: ChainContext):
    raw = ctx.get("mcp_raw", {})
    response = {
        "summary": f"Fetched from MCP: {raw}",
        "query": ctx.get("mcp_query"),
    }
    ctx.set("mcp_response", response)
    return response


@forge_mcp.chain(name="mcp_chain", description="Custom chain backed by MCP")
class MCPChain:
    steps = ["ingest_request", "fetch_from_mcp", "build_custom_response"]


## 10) Run the MCP-backed chain

Pass your own query payload. Cleanup releases the MCP connection and any registered resources.


In [ ]:
async def run_mcp_chain():
    result = await forge_mcp.run(
        "mcp_chain",
        initial_data={"mcp_query": "search Apple earnings"},
    )

    print(f"Success: {result['success']}")
    print(result["context"]["data"].get("mcp_response"))

    await forge_mcp.cleanup_resources()


asyncio.run(run_mcp_chain())


## Next steps

- Swap in your own step logic or data agents and rerun the notebook
- Use `forge.register_resource` to inject shared clients (LLMs, DBs, caches)
- Point the MCP example at your server/tooling and customize the response builder
- Run the same chains via CLI: `flowforge run demo_chain --module user`
- Explore `examples/` and `flowforge/USER_GUIDE.md` for more patterns
